In [1]:
subject_id: int = 3
low_cut_hz: float = 4.
high_cut_hz: float = 38.
n_epochs: int = 3

In [2]:
import pooch
import pooch.utils

In [3]:
def download_patch(*args,fname=None,path=None, **kwargs):
    path = pooch.utils.cache_location(path, env=None, version=None)
    full_path = path.resolve() / fname
    print('Not downloading to avoid side effects')
    return str(full_path)

pooch.retrieve = download_patch

In [6]:
from braindecode.datasets.moabb import MOABBDataset

dataset = MOABBDataset(dataset_name="Schirrmeister2017", subject_ids=[subject_id])

Tensorflow not install, you could not use those pipelines


/opt/conda/envs/demo-env/lib/python3.9/site-packages/moabb/datasets/download.py:54: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_SCHIRRMEISTER2017_PATH"
  set_config(key, get_config("MNE_DATA"))


Not downloading to avoid side effects
Not downloading to avoid side effects
Extracting EDF parameters from /home/jovyan/mne_data/MNE-schirrmeister2017-data/robintibor/high-gamma-dataset/raw/master/data/train/3.edf...
EDF file detected
Channel 'EEG Fp1' recognized as type EEG (renamed to 'Fp1').
Channel 'EEG Fp2' recognized as type EEG (renamed to 'Fp2').
Channel 'EEG Fpz' recognized as type EEG (renamed to 'Fpz').
Channel 'EEG F7' recognized as type EEG (renamed to 'F7').
Channel 'EEG F3' recognized as type EEG (renamed to 'F3').
Channel 'EEG Fz' recognized as type EEG (renamed to 'Fz').
Channel 'EEG F4' recognized as type EEG (renamed to 'F4').
Channel 'EEG F8' recognized as type EEG (renamed to 'F8').
Channel 'EEG FC5' recognized as type EEG (renamed to 'FC5').
Channel 'EEG FC1' recognized as type EEG (renamed to 'FC1').
Channel 'EEG FC2' recognized as type EEG (renamed to 'FC2').
Channel 'EEG FC6' recognized as type EEG (renamed to 'FC6').
Channel 'EEG M1' recognized as type EEG (re

In [ ]:
from braindecode.datautil.preprocess import (
    exponential_moving_standardize, preprocess, Preprocessor)

low_cut_hz = low_cut_hz
high_cut_hz = high_cut_hz

factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(lambda x: x * 1e6),  # Convert from V to uV
    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
                 factor_new=factor_new, init_block_size=init_block_size)
]

preprocess(dataset, preprocessors)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datautil/preprocess.py:10: UserWarning: datautil.preprocess module is deprecated and is now under preprocessing.preprocess, please use from import braindecode.preprocessing.preprocess
  warn('datautil.preprocess module is deprecated and is now under '
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/preprocessing/preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')


In [ ]:
from braindecode.datautil.windowers import create_windows_from_events

trial_start_offset_seconds = -0.5
sfreq = dataset.datasets[0].raw.info['sfreq']
assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

splitted = windows_dataset.split('run')
train_set = splitted['train']
valid_set = splitted['test']

In [ ]:
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 4
# Extract number of chans and time steps from dataset
n_chans = train_set[0][0].shape[0]
input_window_samples = train_set[0][0].shape[1]

model = ShallowFBCSPNet(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto',
)

# Send model to GPU
if cuda:
    print("Using GPU")
    model.cuda()
else:
    print("Using CPU")


In [ ]:
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

from braindecode import EEGClassifier
# These values we found good for shallow network:
lr = 0.0625 * 0.01
weight_decay = 0

# For deep4 they should be:
# lr = 1 * 0.01
# weight_decay = 0.5 * 0.001

batch_size = 64

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(valid_set),  # using valid_set for validation
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)

clf.fit(train_set, y=None, epochs=n_epochs)

In [ ]:
import pandas as pd

# Extract loss and accuracy values for plotting from history object
results_columns = ['train_loss', 'valid_loss', 'train_accuracy', 'valid_accuracy']
df = pd.DataFrame(clf.history[:, results_columns], columns=results_columns,
                  index=clf.history[:, 'epoch'])
df

In [ ]:
import matplotlib.pyplot as mp
mp.figure()
df.plot()

In [ ]:
import os
job_name = os.environ.get('JOB_NAME', None)

if job_name is not None:  
    model_file = f"{job_name}.model.pt"
    model_path = os.path.join(os.getcwd(), model_file)
else:
    model_path = os.path.join(os.getcwd(), 'model.pt')

if not os.path.exists(model_path):
    torch.save(model.state_dict(), model_path)
    print(f"saved model to {model_path}")
else:
    print(f"failed to save model to path {model_path}: file exists")